In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import logging
import shutil
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.config_store import ConfigStore
import mlflow
from src.experiments.sner import sner
from pathlib import Path

import os

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)
logger = logging.getLogger("training")

In [9]:
mlflow.get_tracking_uri()

'https://mlflow-uvl.ifi.uni-heidelberg.de'

In [10]:
sentence_length = 106

run_experiments = os.getenv("UVL_BERT_RUN_EXPERIMENTS", "True") == "True"
pin_commits = os.getenv("UVL_BERT_PIN_COMMITS", "True") == "FALSE"

print(f"{run_experiments=}")
print(f"{pin_commits=}")

run_experiments=True
pin_commits=False


In [14]:
from tooling.config import Experiment, Transformation
#Set Iterations=1 for Split training and testing on datasets. Set dataset to desired dataset split. See tooling.py for split options
base_experiment_config = Experiment(
    name="Base Config 1",
    iterations=1,
    force=False,
    dataset="PKTrainFTest",
    lower_case=True,
)

levels_transformation_config = Transformation(
    description="Levels",
    type="Reduced",
    task="Domain_Level",
    domain_data="Domain_Level",
    activity="Domain_Level",
    stakeholder="Domain_Level",
    system_function="Interaction_Level",
    interaction="Interaction_Level",
    interaction_data="Domain_Level",
    workspace="Interaction_Level",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

label_transformation_config = Transformation(
    description="None",
    type="Full",
    task="Task",
    domain_data="Domain_Data",
    activity="Activity",
    stakeholder="Stakeholder",
    system_function="System_Function",
    interaction="Interaction",
    interaction_data="Interaction_Data",
    workspace="Workspace",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

In [15]:
from tooling.transformation import get_hint_transformation
import pickle

hint_transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(levels_transformation_config)
)

Path("./src/service/models/").mkdir(parents=True, exist_ok=True)

hint_label2id = hint_transformation["label2id"]
pickle.dump(
    hint_label2id, open("./src/service/models/hint_label2id.pickle", "wb")
)
hint_id2label = {y: x for x, y in hint_label2id.items()}
pickle.dump(
    hint_id2label, open("./src/service/models/hint_id2label.pickle", "wb")
)

transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(label_transformation_config)
)
label2id = transformation["label2id"]
pickle.dump(label2id, open("./src/service/models/label2id.pickle", "wb"))
id2label = {y: x for x, y in label2id.items()}
pickle.dump(id2label, open("./src/service/models/id2label.pickle", "wb"))

03:55:19 INFO:Hint Label2Id: hint_label2id={'0': 0, 'System_Level': 1, 'Domain_Level': 2, 'Interaction_Level': 3}
03:55:19 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Task': 1, 'Domain_Data': 2, 'Activity': 3, 'Stakeholder': 4, 'System_Function': 5, 'Interaction': 6, 'Interaction_Data': 7, 'Workspace': 8, 'System_Level': 9}


## Train BiLSTM First Stage Model


In [25]:
from tooling.observability import get_run_id
from tooling.config import BiLSTMConfig, BiLSTM

from copy import deepcopy

bilstm_experiment_config = deepcopy(base_experiment_config)
bilstm_experiment_config.name = "Production"

bilstm_config = BiLSTM(
    batch_size=16,
    learning_rate=0.006,
    number_epochs=5,
    weighted_classes=False,
    sentence_length=106,
)

bilstm_cfg: BiLSTMConfig = OmegaConf.structured(
    BiLSTMConfig(
        bilstm=bilstm_config,
        experiment=bilstm_experiment_config,
        transformation=levels_transformation_config,
    )
)

# bilstm_cfg.experiment.force = True

if run_experiments:
    from experiments.bilstm import bilstm

    bilstm(bilstm_cfg)


bilstm_run_id = get_run_id(bilstm_cfg, pin_commit=pin_commits)

print(bilstm_run_id)
print(mlflow.get_artifact_uri())

bilstm_run = mlflow.get_run(bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bilstm")

04:00:10 INFO:
bilstm:
  type: BiLSTM
  sentence_length: 106
  batch_size: 16
  number_epochs: 5
  verbose: 1
  weighted_classes: false
  learning_rate: 0.006
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 5
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

04:00:11 WARNING:Experiment was already run, aborting


97f6d3072d1644f6a08c33ebc3c165b5
mlflow-artifacts:/6/c2b199bb34b04b278480a86e58086f0c/artifacts


PosixPath('src/service/models/bilstm')

## Train SNER First Stage Model


In [15]:
from tooling.observability import get_run_id
from tooling.config import SNERConfig, SNER

from copy import deepcopy

sner_experiment_config = deepcopy(base_experiment_config)
sner_experiment_config.name = "Production"

sner_config = SNER()

sner_cfg = OmegaConf.structured(
    SNERConfig(
        sner=sner_config,
        experiment=sner_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.sner import sner

    sner(OmegaConf.create(sner_cfg))

sner_run_id = get_run_id(sner_cfg, pin_commit=pin_commits)
print(mlflow.get_artifact_uri())
print(sner_run_id)

sner_run = mlflow.get_run(sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{sner_run.info.artifact_uri}/0_model.ser.gz",
    dst_path=Path("./src/service/models/"),
)
try:
    Path("./src/service/models/sner.ser.gz").unlink()
except FileNotFoundError:
    pass
Path("./src/service/models/0_model.ser.gz").rename(
    "./src/service/models/sner.ser.gz"
)

10:07:01 INFO:
sner:
  type: SNER
experiment:
  name: Production 1
  description: ''
  random_state: 125
  folds: 5
  iterations: 5
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

10:07:01 INFO:New experiment. Running
10:07:01 INFO:Entering mlflow context
10:07:02 INFO:Importing dataset: anno from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/forum/anno_test.json
10:07:02 INFO:Importing dataset: anno from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/forum/anno_train.json
10:07:02 INFO:Imp

AttributeError: 'tuple' object has no attribute 'tb_frame'

## Train BERT First Stage Model


In [10]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_1_experiment_config = deepcopy(base_experiment_config)
bert_1_experiment_config.name = "Production"
bert_1_experiment_config.force = False

bert_1_config = BERT(
    max_len=123,
    number_epochs=8,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=True,
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.0005,
    validation_batch_size=64,
)


bert_1_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_1_config,
        experiment=bert_1_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_1_cfg))

bert_1_run_id = get_run_id(bert_1_cfg, pin_commit=pin_commits)

print(bert_1_run_id)

bert_1_run = mlflow.get_run(bert_1_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_1_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_1"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_1")
## Train BERT Second Stage Model for BERT First Stage

12:34:29 INFO:
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 64
  number_epochs: 8
  learning_rate_bert: 3.0e-05
  learning_rate_classifier: 0.0005
  weight_decay: 0.01
  weighted_classes: true
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 2
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Domain_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

12:34:30 INFO:New experiment. Running
12:34:30 INFO:Entering mlflow context
12:34:31 INFO:Importing dataset: anno from E:\BERT\uvl-tore-class

Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

12:45:35 INFO:Logged iteration result res.precision=0.7138472938976229 res.recall=0.8025448298624706


{'eval_loss': 0.5497260689735413, 'eval_step': 0, 'eval_precision': 0.7138472938976229, 'eval_recall': 0.8025448298624706, 'eval_f1': 0.7556019924559958, 'eval_label_count': 4, 'eval_runtime': 51.2529, 'eval_samples_per_second': 12.136, 'eval_steps_per_second': 0.195, 'epoch': 1.0}
{'loss': 0.5543, 'learning_rate': 2.397106109324759e-05, 'epoch': 1.61}


...

03:25:17 INFO:Logged iteration result res.precision=0.801830583345442 res.recall=0.8088674021556695


{'eval_loss': 1.2070151567459106, 'eval_step': 14, 'eval_precision': 0.801830583345442, 'eval_recall': 0.8088674021556695, 'eval_f1': 0.8053336215203768, 'eval_label_count': 4, 'eval_runtime': 42.9306, 'eval_samples_per_second': 14.489, 'eval_steps_per_second': 0.233, 'epoch': 8.0}
{'train_runtime': 5006.5369, 'train_samples_per_second': 3.976, 'train_steps_per_second': 0.497, 'train_loss': 0.19984252774830416, 'epoch': 8.0}


03:26:04 INFO:Logged iteration result res.precision=0.801830583345442 res.recall=0.8088674021556695
03:26:06 INFO:Logged iteration result res.precision=0.801830583345442 res.recall=0.8088674021556695
03:26:06 INFO:Finished iteration=1
03:26:06 INFO:Logging model artifact (might take a while)
03:30:53 INFO:Breaking early after iteration=1 of 5 folds
03:31:05 INFO:Logged experiment result res.mean_precision=0.7955724672064002 res.mean_recall=0.8054568108482079
03:31:06 INFO:Left mlflow context


32c719a647af445394273d5e44d9f60d


WindowsPath('src/service/models/bert_1')

In [11]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_bert_experiment_config = deepcopy(base_experiment_config)
bert_2_bert_experiment_config.name = "Production"

bert_2_bert_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=5,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
    validation_batch_size=64,
)

cfg = DualModelStagedBERTConfig(
    bert=bert_2_bert_config,
    experiment=bert_2_bert_experiment_config,
    transformation=label_transformation_config,
    first_model_bert=bert_1_cfg,
)

bert_2_bert_cfg: DualModelStagedBERTConfig = OmegaConf.structured(cfg)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_bert_cfg))

bert_2_bert_run_id = get_run_id(bert_2_bert_cfg, pin_commit=pin_commits)

print(bert_2_bert_run_id)

bert_2_bert_run = mlflow.get_run(bert_2_bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bert_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_2_bert")

03:49:34 INFO:
first_model_bert:
  bert:
    model: bert-base-uncased
    type: BERT
    max_len: 123
    train_batch_size: 8
    validation_batch_size: 64
    number_epochs: 8
    learning_rate_bert: 3.0e-05
    learning_rate_classifier: 0.0005
    weight_decay: 0.01
    weighted_classes: true
  experiment:
    name: Production
    description: ''
    random_state: 125
    folds: 5
    iterations: 2
    average: macro
    dataset: all
    lower_case: true
    force: false
    pin_commit: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Domain_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
    system_level: System_Level
first_model_bilstm: null
first_model_sner: null


Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

03:51:06 INFO:Loading Model
03:51:07 INFO:Creating hint column


Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

...

05:51:25 INFO:Logged iteration result res.precision=0.72517549671798 res.recall=0.7263698817257245


{'eval_loss': 2.806786298751831, 'eval_step': 8, 'eval_precision': 0.72517549671798, 'eval_recall': 0.7263698817257245, 'eval_f1': 0.7257721978298614, 'eval_label_count': 8, 'eval_runtime': 46.8813, 'eval_samples_per_second': 13.268, 'eval_steps_per_second': 0.213, 'epoch': 5.0}
{'train_runtime': 3223.4452, 'train_samples_per_second': 3.859, 'train_steps_per_second': 0.482, 'train_loss': 0.506001881694487, 'epoch': 5.0}


05:52:17 INFO:Logged iteration result res.precision=0.72517549671798 res.recall=0.7263698817257245
05:52:19 INFO:Logged iteration result res.precision=0.72517549671798 res.recall=0.7263698817257245
05:52:19 INFO:Finished iteration=1
05:52:19 INFO:Logging model artifact (might take a while)
06:02:46 INFO:Breaking early after iteration=1 of 5 folds
06:02:57 INFO:Logged experiment result res.mean_precision=0.7134253781941458 res.mean_recall=0.7273624374473947
06:02:57 INFO:Left mlflow context


4f62195fb8ff453ca888ca2004629de4


WindowsPath('src/service/models/bert_2_bert')

## Train BERT Second Stage Model for SNER First Stage


In [29]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"

bert_2_sner_experiment_config = deepcopy(base_experiment_config)
bert_2_sner_experiment_config.name = "Production"

bert_2_sner_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=5,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
)

bert_2_sner_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_sner_config,
        experiment=bert_2_sner_experiment_config,
        transformation=label_transformation_config,
        first_model_sner=sner_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_sner_cfg))

bert_2_sner_run_id = get_run_id(bert_2_sner_cfg, pin_commit=pin_commits)

print(bert_2_sner_run_id)

bert_2_sner_run = mlflow.get_run(bert_2_sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_sner_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_sner"))
except FileNotFoundError:
    pass

Path("./src/service/models/0_model").rename("./src/service/models/bert_2_sner")

06:29:10 INFO:
first_model_bert: null
first_model_bilstm: null
first_model_sner:
  sner:
    type: SNER
  experiment:
    name: Production
    description: ''
    random_state: 125
    folds: 5
    iterations: 5
    average: macro
    dataset: all
    lower_case: true
    force: false
    pin_commit: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
    system_level: System_Level
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 3.0e-05
  learning_rate_classifier: 0.16
  weight_decay: 0.1
  weighted_classes: tr

Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

Test
StagedBertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), ep

06:31:09 INFO:Logged iteration result res.precision=0.5216784271253025 res.recall=0.5347241430022265


...

07:20:34 INFO:Logged iteration result res.precision=0.6025761601385351 res.recall=0.537407386595133


{'eval_loss': 3.728144645690918, 'eval_step': 8, 'eval_precision': 0.6025761601385351, 'eval_recall': 0.537407386595133, 'eval_f1': 0.5681290407610347, 'eval_label_count': 10, 'eval_runtime': 5.8172, 'eval_samples_per_second': 106.924, 'eval_steps_per_second': 3.438, 'epoch': 5.0}
{'train_runtime': 503.9214, 'train_samples_per_second': 24.686, 'train_steps_per_second': 3.086, 'train_loss': 0.7389109912025967, 'epoch': 5.0}


07:20:44 INFO:Logged iteration result res.precision=0.6025761601385351 res.recall=0.537407386595133
07:20:45 INFO:Logged iteration result res.precision=0.6025761601385351 res.recall=0.537407386595133
07:20:45 INFO:Finished iteration=4
07:20:45 INFO:Logging model artifact (might take a while)
07:22:41 INFO:Breaking early after iteration=4 of 5 folds
07:22:52 INFO:Logged experiment result res.mean_precision=0.6256756490822082 res.mean_recall=0.5622017690074544
07:22:53 INFO:Left mlflow context


89900f3ccbc94ae2b3566471e8c1cc23


PosixPath('src/service/models/bert_2_sner')

## Train BERT Second Stage Model for BILSTM First Stage


In [30]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bilstm_config = BiLSTM(
    batch_size=16,
    learning_rate=0.006,
    number_epochs=5,
    weighted_classes=False,
    sentence_length=106,
)


bert_2_bilstm_experiment_config = deepcopy(base_experiment_config)
bert_2_bilstm_experiment_config.name = "Production"

bert_2_bilstm_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=5,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
)

bert_2_bilstm_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bilstm_config,
        experiment=bert_2_bilstm_experiment_config,
        transformation=label_transformation_config,
        first_model_bilstm=bilstm_cfg,
    )
)
bert_2_bilstm_cfg.first_model_bilstm.bilstm.sentence_length = 106


if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_bilstm_cfg))

bert_2_bilstm_run_id = get_run_id(bert_2_bilstm_cfg, pin_commit=pin_commits)

print(bert_2_bilstm_run_id)

bert_2_bilstm_run = mlflow.get_run(bert_2_bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename(
    "./src/service/models/bert_2_bilstm"
)

07:24:05 INFO:
first_model_bert: null
first_model_bilstm:
  bilstm:
    type: BiLSTM
    sentence_length: 106
    batch_size: 16
    number_epochs: 5
    verbose: 1
    weighted_classes: false
    learning_rate: 0.006
  experiment:
    name: Production
    description: ''
    random_state: 125
    folds: 5
    iterations: 5
    average: macro
    dataset: all
    lower_case: true
    force: false
    pin_commit: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
    system_level: System_Level
first_model_sner: null
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_ba

07:24:23 INFO:loading projection weights from /Users/bockstaller/gensim-data/glove-twitter-100/glove-twitter-100.gz
07:24:59 INFO:KeyedVectors lifecycle event {'msg': 'loaded (1193514, 100) matrix of type float32 from /Users/bockstaller/gensim-data/glove-twitter-100/glove-twitter-100.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2023-10-14T19:24:59.451313', 'gensim': '4.3.2', 'python': '3.11.4 (main, Jun 20 2023, 19:14:10) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-14.0-arm64-arm-64bit', 'event': 'load_word2vec_format'}
2023-10-14 19:24:59.854811: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-10-14 19:24:59.854864: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-10-14 19:24:59.854868: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-10-14 19:24:59.854938: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of 

78/78 [==============================] - 5s 32ms/step


Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

2023-10-14 19:25:47.669802: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-14 19:25:47.781038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-14 19:25:47.793333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-14 19:25:48.795220: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-14 19:25:48.806596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

08:23:12 INFO:Logged iteration result res.precision=0.531509451110273 res.recall=0.6502450971683986


{'eval_loss': 1.4704921245574951, 'eval_step': 8, 'eval_precision': 0.531509451110273, 'eval_recall': 0.6502450971683986, 'eval_f1': 0.5849123495002153, 'eval_label_count': 10, 'eval_runtime': 6.2953, 'eval_samples_per_second': 98.805, 'eval_steps_per_second': 3.177, 'epoch': 5.0}
{'train_runtime': 510.4063, 'train_samples_per_second': 24.373, 'train_steps_per_second': 3.047, 'train_loss': 1.0884260048820276, 'epoch': 5.0}


08:23:23 INFO:Logged iteration result res.precision=0.531509451110273 res.recall=0.6502450971683986
08:23:24 INFO:Logged iteration result res.precision=0.531509451110273 res.recall=0.6502450971683986
08:23:24 INFO:Finished iteration=4
08:23:24 INFO:Logging model artifact (might take a while)
08:25:20 INFO:Breaking early after iteration=4 of 5 folds
08:25:29 INFO:Logged experiment result res.mean_precision=0.5521306671186457 res.mean_recall=0.6695003356234419
08:25:30 INFO:Left mlflow context


b85cc01c9dce4c9fa2072f0c4b2bf6fa


PosixPath('src/service/models/bert_2_bilstm')

## Train BERT E2E Model


In [16]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_experiment_config = deepcopy(base_experiment_config)
bert_experiment_config.name = "Production"

bert_config = BERT(
    learning_rate_bert=6e-05,
    learning_rate_classifier=0.1,
    max_len=123,
    number_epochs=8,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=False,
)

bert_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_config,
        experiment=bert_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_cfg))

bert_run_id = get_run_id(bert_cfg, pin_commit=pin_commits)

print(bert_run_id)

run = mlflow.get_run(bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert")

03:55:25 INFO:
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 32
  number_epochs: 8
  learning_rate_bert: 6.0e-05
  learning_rate_classifier: 0.1
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: PKTrainFTest
  lower_case: true
  force: false
  pin_commit: false
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  activity: Activity
  stakeholder: Stakeholder
  system_function: System_Function
  interaction: Interaction
  interaction_data: Interaction_Data
  workspace: Workspace
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

03:55:27 INFO:New experiment. Running
03:55:27 INFO:Entering mlflow context
03:55:27 INFO:Importing dataset: prolific from E:\BERT\uvl-tore-classifier-bert\src\data\

prolific
komoot
anno
                                sentence_id sentence_idx   string  \
0      f162658f-bc5a-11ee-a8ec-34e12df9aeb0            0     this   
1      f162658f-bc5a-11ee-a8ec-34e12df9aeb0            1      app   
2      f162658f-bc5a-11ee-a8ec-34e12df9aeb0            2       is   
3      f162658f-bc5a-11ee-a8ec-34e12df9aeb0            3      for   
4      f162658f-bc5a-11ee-a8ec-34e12df9aeb0            4  mapping   
...                                     ...          ...      ...   
41481  f2c63b47-bc5a-11ee-b11d-34e12df9aeb0           14    found   
41482  f2c63b47-bc5a-11ee-b11d-34e12df9aeb0           15       it   
41483  f2c63b47-bc5a-11ee-b11d-34e12df9aeb0           16        .   
41484  f2c63b48-bc5a-11ee-b461-34e12df9aeb0            0    grrrr   
41485  f2c63b48-bc5a-11ee-b461-34e12df9aeb0            1    .....   

            tore_label  
0                    0  
1         System_Level  
2                    0  
3                    0  
4      System_Function  


03:56:45 INFO:Created datasets for foldless run
03:56:45 INFO:Starting iteration=1


Map:   0%|          | 0/2245 [00:00<?, ? examples/s]

Map:   0%|          | 0/865 [00:00<?, ? examples/s]

...

{'eval_loss': 1.7807481288909912, 'eval_step': 12, 'eval_precision': 0.3804349774912863, 'eval_recall': 0.45406257954092644, 'eval_f1': 0.4140007020311018, 'eval_label_count': 10, 'eval_runtime': 69.1592, 'eval_samples_per_second': 12.507, 'eval_steps_per_second': 0.405, 'epoch': 7.0}
{'loss': 0.0332, 'learning_rate': 6.619217081850534e-06, 'epoch': 7.12}


05:54:54 INFO:Logged iteration result res.precision=0.3646061997385896 res.recall=0.45250910303452196


{'eval_loss': 1.9154365062713623, 'eval_step': 14, 'eval_precision': 0.3646061997385896, 'eval_recall': 0.45250910303452196, 'eval_f1': 0.4038294812117772, 'eval_label_count': 10, 'eval_runtime': 72.7276, 'eval_samples_per_second': 11.894, 'eval_steps_per_second': 0.385, 'epoch': 8.0}
{'train_runtime': 7091.4378, 'train_samples_per_second': 2.533, 'train_steps_per_second': 0.317, 'train_loss': 0.2415243539945935, 'epoch': 8.0}


05:56:18 INFO:Logged iteration result res.precision=0.3646061997385896 res.recall=0.45250910303452196
05:56:20 INFO:Logged iteration result res.precision=0.3646061997385896 res.recall=0.45250910303452196
05:56:20 INFO:Finished iteration=1
05:56:20 INFO:Logging model artifact (might take a while)
05:57:24 ERROR:API request to https://mlflow-uvl.ifi.uni-heidelberg.de/api/2.0/mlflow-artifacts/artifacts/6/4d6689f4de2b4b8597d990f413242c49/artifacts/confusion_matrix.png failed with exception HTTPSConnectionPool(host='mlflow-uvl.ifi.uni-heidelberg.de', port=443): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/6/4d6689f4de2b4b8597d990f413242c49/artifacts/confusion_matrix.png (Caused by ResponseError('too many 500 error responses'))


MlflowException: API request to https://mlflow-uvl.ifi.uni-heidelberg.de/api/2.0/mlflow-artifacts/artifacts/6/4d6689f4de2b4b8597d990f413242c49/artifacts/confusion_matrix.png failed with exception HTTPSConnectionPool(host='mlflow-uvl.ifi.uni-heidelberg.de', port=443): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/6/4d6689f4de2b4b8597d990f413242c49/artifacts/confusion_matrix.png (Caused by ResponseError('too many 500 error responses'))